# Quality features treatment

In [1]:
import pandas as pd
import numpy as np

# 1. Carregar o dataset
print("\nCarregando dataset 01_data_collection_and_integration.csv...")
try:
    df = pd.read_csv("datasets/01_data_collection_and_integration.csv")
    print(f"Dataset carregado: {df.shape[0]} linhas, {df.shape[1]} colunas")
except FileNotFoundError:
    print("Arquivo data_collection_and_integration.csv não encontrado.")
    raise Exception("Arquivo de dados não encontrado")

# 2. Verificar se as colunas de qualidade já foram processadas
print("\nVerificando status das colunas de qualidade...")

# Verificar se as colunas consolidadas já existem
has_numeric = 'qualidade_numerica' in df.columns
has_textual = 'qualidade_textual' in df.columns

# Listar todas as variantes das colunas de qualidade
quality_variants = [
    'Qualidade (Nome)', 'Qualidade (Número)', 'Qualidade (Numero)', 
    'Qualidade (nome)', 'Qualidade (número)', 'Qualidade (Nombre)', 
    'Qualidade', 'Qualidade (Número) ', 'Qualidade (Nome) '
]

# Checar quais variantes existem no dataset
existing_variants = [col for col in quality_variants if col in df.columns]
print(f"Variantes de 'Qualidade' encontradas: {len(existing_variants)}")
for col in existing_variants:
    print(f"  - {col}: {df[col].notna().sum()} valores não-nulos")

# 3. Determinar próximos passos com base na análise
if len(existing_variants) == 0 and not (has_numeric or has_textual):
    print("\nNenhuma coluna de qualidade encontrada no dataset.")
    
    # Verificar se talvez as colunas foram renomeadas
    possible_renamed = [col for col in df.columns if 'qual' in col.lower()]
    if possible_renamed:
        print("Possíveis colunas relacionadas encontradas:")
        for col in possible_renamed:
            print(f"  - {col}")
    
    print("\nVamos criar colunas vazias para qualidade_numerica e qualidade_textual.")
    # Criar colunas vazias
    df['qualidade_numerica'] = np.nan
    df['qualidade_textual'] = None
    
elif has_numeric and has_textual:
    print("\nAs colunas consolidadas já existem:")
    print(f"  - qualidade_numerica: {df['qualidade_numerica'].notna().sum()} valores não-nulos")
    print(f"  - qualidade_textual: {df['qualidade_textual'].notna().sum()} valores não-nulos")
    
    # Remover as colunas originais se ainda existirem
    if existing_variants:
        print("\nRemovendo colunas originais redundantes...")
        df = df.drop(columns=existing_variants)
        print(f"  - {len(existing_variants)} colunas originais removidas")
    
else:
    # Processar e consolidar as colunas de qualidade
    print("\nConsolidando colunas de qualidade...")
    
    # Separar colunas numéricas e textuais
    numeric_cols = []
    text_cols = []
    
    for col in existing_variants:
        if 'nome' in col.lower() or 'name' in col.lower() or 'nombre' in col.lower():
            text_cols.append(col)
        else:
            # Tentar converter para confirmar se é numérica
            try:
                numeric_test = pd.to_numeric(df[col], errors='coerce')
                # Se pelo menos 50% dos valores não-nulos converteram, considerar numérica
                not_null = df[col].notna().sum()
                if not_null > 0 and numeric_test.notna().sum() / not_null >= 0.5:
                    numeric_cols.append(col)
                else:
                    text_cols.append(col)
            except:
                text_cols.append(col)
    
    # Consolidar colunas numéricas
    if numeric_cols:
        print(f"\nConsolidando {len(numeric_cols)} colunas numéricas:")
        for col in numeric_cols:
            print(f"  - {col}")
        
        df['qualidade_numerica'] = np.nan
        # Preencher em ordem de prioridade (mais valores não-nulos primeiro)
        numeric_cols_sorted = sorted(numeric_cols, key=lambda x: df[x].notna().sum(), reverse=True)
        
        for col in numeric_cols_sorted:
            mask = df['qualidade_numerica'].isna() & df[col].notna()
            if mask.sum() > 0:
                df.loc[mask, 'qualidade_numerica'] = pd.to_numeric(df.loc[mask, col], errors='coerce')
                print(f"    Preenchendo {mask.sum()} valores de '{col}'")
        
        print(f"  - qualidade_numerica criada com {df['qualidade_numerica'].notna().sum()} valores não-nulos")
    
    # Consolidar colunas textuais
    if text_cols:
        print(f"\nConsolidando {len(text_cols)} colunas textuais:")
        for col in text_cols:
            print(f"  - {col}")
        
        df['qualidade_textual'] = None
        # Preencher em ordem de prioridade
        text_cols_sorted = sorted(text_cols, key=lambda x: df[x].notna().sum(), reverse=True)
        
        for col in text_cols_sorted:
            mask = df['qualidade_textual'].isna() & df[col].notna()
            if mask.sum() > 0:
                df.loc[mask, 'qualidade_textual'] = df.loc[mask, col]
                print(f"    Preenchendo {mask.sum()} valores de '{col}'")
        
        print(f"  - qualidade_textual criada com {df['qualidade_textual'].notna().sum()} valores não-nulos")
    
    # Remover as colunas originais
    print("\nRemovendo colunas originais...")
    df = df.drop(columns=existing_variants)
    print(f"  - {len(existing_variants)} colunas originais removidas")

# 4. Salvar o dataset atualizado
print("\nSalvando dataset atualizado...")
df.to_csv("datasets/01_data_collection_and_integration.csv", index=False)
print(f"Dataset salvo como '01_data_collection_and_integration.csv' com {df.shape[1]} colunas")
print("  - Inclui apenas 'qualidade_numerica' e 'qualidade_textual' como colunas de qualidade")

# 5. Mostrar as primeiras linhas para verificação
print("\nPrimeiras linhas do dataset atualizado:")
print(df.head())


Carregando dataset 01_data_collection_and_integration.csv...


/var/tmp/ipykernel_8434/925404849.py:7: DtypeWarning: Columns (18,21,23,24,26,42) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datasets/01_data_collection_and_integration.csv")


Dataset carregado: 138820 linhas, 48 colunas

Verificando status das colunas de qualidade...
Variantes de 'Qualidade' encontradas: 8
  - Qualidade (Nome): 49143 valores não-nulos
  - Qualidade (Número): 17846 valores não-nulos
  - Qualidade (Numero): 31297 valores não-nulos
  - Qualidade (nome): 19796 valores não-nulos
  - Qualidade (número): 19796 valores não-nulos
  - Qualidade: 43392 valores não-nulos
  - Qualidade (Número) : 17597 valores não-nulos
  - Qualidade (Nome) : 17597 valores não-nulos

Consolidando colunas de qualidade...

Consolidando 4 colunas numéricas:
  - Qualidade (Número)
  - Qualidade (Numero)
  - Qualidade (número)
  - Qualidade (Número) 
    Preenchendo 31297 valores de 'Qualidade (Numero)'
    Preenchendo 19796 valores de 'Qualidade (número)'
    Preenchendo 17846 valores de 'Qualidade (Número)'
    Preenchendo 17597 valores de 'Qualidade (Número) '
  - qualidade_numerica criada com 86536 valores não-nulos

Consolidando 4 colunas textuais:
  - Qualidade (Nome)


# Data Cleaning, Preprocessing
1. Handle missing values
2. Remove or treat outliers
3. Normalize/standardize values
4. Remove duplicates

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# 1 - Carregar dataset
print("Carregando dataset...")
df = pd.read_csv("datasets/01_data_collection_and_integration.csv")
print(f"Dataset carregado: {df.shape[0]} linhas, {df.shape[1]} colunas")

# 2 - Remoção seletiva de duplicatas (preservando compradores)
print("Tratamento de duplicatas...")
duplicated_emails = df.duplicated('email_norm', keep=False)
duplicate_count = duplicated_emails.sum()
print(f"Identificados {duplicate_count} registros com emails duplicados")

# Separar compradores antes do processamento
buyers = df[df['target'] == 1].copy()
non_buyers = df[df['target'] == 0].copy()
print(f"Separados {len(buyers)} compradores e {len(non_buyers)} não-compradores")

# Remover apenas duplicatas entre não-compradores
print("Removendo apenas duplicatas entre não-compradores...")
non_buyers_dedup = non_buyers.sort_values('Marca temporal').drop_duplicates(subset=['email_norm'], keep='first')
print(f"Removidas {len(non_buyers) - len(non_buyers_dedup)} duplicatas de não-compradores")

# Recombinar dataset
df = pd.concat([buyers, non_buyers_dedup], ignore_index=True)
print(f"Dataset após tratamento de duplicatas: {df.shape[0]} linhas, {df.shape[1]} colunas")

# 3 - Verificar valores ausentes
print("\nAnalisando valores ausentes...")
missing_counts = df.isnull().sum()
missing_pct = (missing_counts / len(df)) * 100

print("Top 10 colunas com mais valores ausentes:")
missing_df = pd.DataFrame({'Count': missing_counts, 'Percentage': missing_pct})
missing_df = missing_df.sort_values('Percentage', ascending=False)
for col, row in missing_df.head(10).iterrows():
    print(f"  {col}: {row['Count']} ausentes ({row['Percentage']:.2f}%)")

# 4 - Estratégias de tratamento para valores ausentes
print("\nAplicando estratégias para valores ausentes...")

# 4.1 - Colunas de alta ausência (>95%) - remover (elevamos o limite para 95%)
high_missing_cols = missing_df[missing_df['Percentage'] > 95].index.tolist()
if high_missing_cols:
    print(f"Removendo {len(high_missing_cols)} colunas com >95% ausência:")
    for col in high_missing_cols[:5]:  # Mostrar apenas as 5 primeiras para concisão
        print(f"  {col}: {missing_df.loc[col, 'Percentage']:.2f}% ausentes")
    if len(high_missing_cols) > 5:
        print(f"  ... e {len(high_missing_cols) - 5} outras colunas")
    df = df.drop(columns=high_missing_cols)

# 4.2 - Colunas UTM (tratar para análise de marketing)
utm_cols = [col for col in df.columns if col.startswith('UTM_') or 'utm' in col.lower()]
for col in utm_cols:
    if col in df.columns:
        df[col] = df[col].fillna('unknown')
        print(f"  Preenchidos valores ausentes em '{col}' com 'unknown'")

# 4.3 - Dados categóricos (preencher com 'desconhecido')
cat_cols = [
    '¿Cuál es tu género?', '¿Cuál es tu edad?', '¿Cual es tu país?',
    '¿Hace quánto tiempo me conoces?', '¿Cuál es tu disponibilidad de tiempo para estudiar inglés?',
    '¿Cuál es tu sueldo anual? (en dólares)', '¿Cuánto te gustaría ganar al año?',
    '¿Crees que aprender inglés te acercaría más al salario que mencionaste anteriormente?',
    '¿Crees que aprender inglés puede ayudarte en el trabajo o en tu vida diaria?',
    'Qualidade', 'lançamento'
]

for col in cat_cols:
    if col in df.columns:
        na_count = df[col].isna().sum()
        if na_count > 0:
            df[col] = df[col].fillna('desconhecido')
            print(f"  Preenchidos {na_count} valores ausentes em '{col}' com 'desconhecido'")

# 4.4 - Colunas de texto livre (preencher com string vazia)
text_cols = [
    'Cuando hables inglés con fluidez, ¿qué cambiará en tu vida? ¿Qué oportunidades se abrirán para ti?',
    '¿Qué esperas aprender en la Semana de Cero a Inglés Fluido?',
    'Déjame un mensaje',
    '¿Qué esperas aprender en la Inmersión Desbloquea Tu Inglés En 72 horas?'
]

for col in text_cols:
    if col in df.columns:
        na_count = df[col].isna().sum()
        if na_count > 0:
            df[col] = df[col].fillna('')
            print(f"  Preenchidos {na_count} valores ausentes em '{col}' com string vazia")

# 4.5 - Colunas de qualidade (tratamento específico)
# Primeiro identificar colunas de qualidade que são realmente numéricas
quality_cols = [col for col in df.columns if 'qualidade' in col.lower() or 'qualidad' in col.lower()]
quality_numeric_cols = []

# Verificar quais colunas de qualidade são numéricas
for col in quality_cols:
    if col in df.columns:
        # Tentar converter para numérico
        try:
            # Verificar se os valores não nulos podem ser convertidos para numérico
            non_null_values = df[col].dropna()
            pd.to_numeric(non_null_values, errors='raise')
            quality_numeric_cols.append(col)
        except:
            print(f"  Coluna '{col}' contém valores não numéricos, será tratada como categórica")
            # Se não for numérica, tratar como categórica
            if df[col].isna().sum() > 0:
                df[col] = df[col].fillna('desconhecido')

# Processar apenas colunas de qualidade verdadeiramente numéricas
for col in quality_numeric_cols:
    if col in df.columns:
        # Converter para numérico forçando valores não numéricos para NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
        na_count = df[col].isna().sum()
        if na_count > 0:
            # Usar mediana global
            median_value = df[col].median()
            df[col] = df[col].fillna(median_value)
            print(f"  Preenchidos {na_count} valores ausentes em '{col}' com mediana global ({median_value:.2f})")

# 4.6 - Outras colunas numéricas (preencher com mediana)
other_numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
other_numeric_cols = [col for col in other_numeric_cols 
                     if col not in ['target'] + quality_numeric_cols]

for col in other_numeric_cols:
    if col in df.columns:
        na_count = df[col].isna().sum()
        if na_count > 0:
            df[col] = df[col].fillna(df[col].median())
            print(f"  Preenchidos {na_count} valores ausentes em '{col}' com mediana ({df[col].median()})")

# 5 - Tratamento de Outliers (mais conservador para colunas de qualidade)
print("\nIdentificando e tratando outliers...")
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['target']]

for col in numeric_cols:
    if col in df.columns and df[col].nunique() > 10:  # Apenas colunas com variabilidade suficiente
        # Tratamento especial para colunas de qualidade (tratamento menos agressivo)
        if col in quality_numeric_cols:
            # Usar percentis 1 e 99 para preservar mais variabilidade
            lower_bound = df[col].quantile(0.01)
            upper_bound = df[col].quantile(0.99)
            
            # Contar outliers
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
            outlier_count = len(outliers)
            
            if outlier_count > 0:
                print(f"  '{col}': {outlier_count} outliers identificados ({outlier_count/len(df):.2%})")
                
                # Aplicar capping para valores extremos
                df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
                print(f"    Aplicado capping conservador (limite inferior: {lower_bound:.2f}, superior: {upper_bound:.2f})")
        else:
            # Tratamento padrão para outras colunas numéricas
            q1 = df[col].quantile(0.25)
            q3 = df[col].quantile(0.75)
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            
            # Contar outliers
            outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]
            outlier_count = len(outliers)
            
            if outlier_count > 0:
                print(f"  '{col}': {outlier_count} outliers identificados ({outlier_count/len(df):.2%})")
                
                # Aplicar capping para valores extremos
                df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
                print(f"    Aplicado capping (limite inferior: {lower_bound:.2f}, superior: {upper_bound:.2f})")

# 6 - Normalização de valores
print("\nNormalizando valores numéricos...")
# Selecionar colunas numéricas que não são target
cols_to_normalize = [col for col in numeric_cols if col != 'target' and col in df.columns]

if cols_to_normalize:
    # Criar cópia para preservar os dados originais
    df_norm = df.copy()
    
    # Normalizar usando StandardScaler
    scaler = StandardScaler()
    # Verificar se ainda tem variabilidade após o tratamento de outliers
    cols_with_std = []
    for col in cols_to_normalize:
        if df[col].std() > 0:
            cols_with_std.append(col)
    
    if cols_with_std:
        df_norm[cols_with_std] = scaler.fit_transform(df[cols_with_std])
    
    print(f"Normalizadas {len(cols_with_std)} colunas numéricas com variabilidade suficiente:")
    for col in cols_with_std[:5]:  # Mostrar apenas as 5 primeiras para concisão
        print(f"  {col}: média={df[col].mean():.2f}, std={df[col].std():.2f} → média={df_norm[col].mean():.2f}, std={df_norm[col].std():.2f}")
    if len(cols_with_std) > 5:
        print(f"  ... e {len(cols_with_std) - 5} outras colunas")
    
    # Substituir o DataFrame original pelo normalizado
    df = df_norm

# 7 - Converter tipos de dados
print("\nConvertendo tipos de dados...")
# Data e hora
if 'Marca temporal' in df.columns:
    df['Marca temporal'] = pd.to_datetime(df['Marca temporal'], errors='coerce')
    print("  'Marca temporal' convertido para datetime")
if 'DATA' in df.columns:
    df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce')
    print("  'DATA' convertido para datetime")

# 8 - Verificar valores pós-processamento
print("\nResumo do dataset pós-processamento:")
print(f"Dimensões: {df.shape[0]} linhas, {df.shape[1]} colunas")
missing_counts = df.isnull().sum().sum()
print(f"Valores ausentes restantes: {missing_counts}")
if missing_counts > 0:
    print("Detalhes dos valores ausentes restantes:")
    for col, count in df.isnull().sum().items():
        if count > 0:
            print(f"  {col}: {count} valores ausentes")

# 9 - Verificar distribuição da variável target
print("\nDistribuição da variável target:")
if 'target' in df.columns:
    value_counts = df['target'].value_counts()
    print(f"  0 (não converteu): {value_counts.get(0, 0)} ({value_counts.get(0, 0)/len(df):.2%})")
    print(f"  1 (converteu): {value_counts.get(1, 0)} ({value_counts.get(1, 0)/len(df):.2%})")

# 10 - Estatísticas por lançamento
if 'lançamento' in df.columns and 'target' in df.columns:
    print("\nEstatísticas por lançamento:")
    # Criar uma tabela pivô para exibir estatísticas sem depender de groupby.median()
    launch_stats = pd.DataFrame({
        'registros': df.groupby('lançamento')['target'].count(),
        'conversoes': df.groupby('lançamento')['target'].sum()
    })
    launch_stats['taxa_conversao'] = (launch_stats['conversoes'] / launch_stats['registros']) * 100
    
    for launch, row in launch_stats.iterrows():
        print(f"  {launch}: {int(row['registros'])} registros, {int(row['conversoes'])} conversões ({row['taxa_conversao']:.2f}%)")

# 11 - Salvar dataset limpo
df.to_csv("datasets/02_1_data_cleaned_and_preprocessed.csv", index=False)
print("\nDataset limpo salvo em 'datasets/02_1_data_cleaned_and_preprocessed.csv'")

Carregando dataset...
Dataset carregado: 138820 linhas, 42 colunas
Tratamento de duplicatas...
Identificados 47832 registros com emails duplicados
Separados 1880 compradores e 136940 não-compradores
Removendo apenas duplicatas entre não-compradores...
Removidas 32207 duplicatas de não-compradores
Dataset após tratamento de duplicatas: 106613 linhas, 42 colunas

Analisando valores ausentes...
Top 10 colunas com mais valores ausentes:
  UTM_CAMPAIGN(CORRECTA): 106613.0 ausentes (100.00%)
  Unnamed: 12: 106613.0 ausentes (100.00%)
  Unnamed: 11: 106613.0 ausentes (100.00%)
  Unnamed: 9: 106613.0 ausentes (100.00%)
  Unnamed: 8: 106613.0 ausentes (100.00%)
  Unnamed: 10: 105644.0 ausentes (99.09%)
  UTM_CAMPAIGN (VINI): 96190.0 ausentes (90.22%)
  UTM_CAMPAIGN2: 94410.0 ausentes (88.55%)
  ¿Qué esperas aprender en la Inmersión Desbloquea Tu Inglés En 72 horas?: 92615.0 ausentes (86.87%)
  GCLID: 90053.0 ausentes (84.47%)

Aplicando estratégias para valores ausentes...
Removendo 6 colunas c

# Feature treatment (categorical and numerical)
1. Do the treatment specified at EDA (column_treatment.txt) - Except the text features

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 1 - Carregar dados pré-processados
print("Carregando dataset pré-processado...")
df = pd.read_csv("datasets/02_1_data_cleaned_and_preprocessed.csv")
print(f"Dataset carregado: {df.shape[0]} linhas, {df.shape[1]} colunas")

# 2 - Verificar e remover colunas irrelevantes iniciais
print("\nRemovendo colunas irrelevantes (exceto texto)...")
# Colunas que devem ser removidas conforme especificação
cols_to_remove = [
    '¿Cómo te llamas?',  # Substituída por feature de comprimento
    '¿Cual es tu telefono?',  # Substituída por indicador de validade
    '¿Cuál es tu instagram?'  # Substituída por indicador de username válido
]

# Verificar quais colunas existem no dataframe
cols_to_remove = [col for col in cols_to_remove if col in df.columns]

# Criar as features derivadas antes de remover as originais
print("Criando features derivadas (não textuais)...")

# 3 - Features de identidade
if '¿Cómo te llamas?' in df.columns:
    df['name_length'] = df['¿Cómo te llamas?'].str.len()
    df['name_word_count'] = df['¿Cómo te llamas?'].str.split().str.len()

if '¿Cual es tu telefono?' in df.columns:
    # Verificar se o telefone tem pelo menos 8 dígitos
    df['valid_phone'] = df['¿Cual es tu telefono?'].str.replace(r'\D', '', regex=True).str.len() >= 8

if '¿Cuál es tu instagram?' in df.columns:
    df['has_instagram'] = df['¿Cuál es tu instagram?'].notna() & (df['¿Cuál es tu instagram?'] != '')

# 4 - Features de tempo (Marca temporal)
if 'Marca temporal' in df.columns:
    # Converter para datetime se ainda não for
    if not pd.api.types.is_datetime64_dtype(df['Marca temporal']):
        df['Marca temporal'] = pd.to_datetime(df['Marca temporal'], errors='coerce')
    
    # Extrair componentes básicos
    df['hour'] = df['Marca temporal'].dt.hour
    df['day_of_week'] = df['Marca temporal'].dt.dayofweek
    df['month'] = df['Marca temporal'].dt.month
    df['year'] = df['Marca temporal'].dt.year
    
    # Features cíclicas para hora e dia da semana
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    # Período do dia
    df['period_of_day'] = pd.cut(
        df['hour'], 
        bins=[0, 6, 12, 18, 24], 
        labels=['madrugada', 'manha', 'tarde', 'noite']
    )

# Repetir processo para DATA se existir
if 'DATA' in df.columns:
    if not pd.api.types.is_datetime64_dtype(df['DATA']):
        df['DATA'] = pd.to_datetime(df['DATA'], errors='coerce')
    
    # Extrair componentes apenas se conversão foi bem-sucedida
    if not df['DATA'].isna().all():
        df['utm_hour'] = df['DATA'].dt.hour
        df['utm_day_of_week'] = df['DATA'].dt.dayofweek
        df['utm_month'] = df['DATA'].dt.month
        df['utm_year'] = df['DATA'].dt.year

# 5 - Remover as colunas originais (exceto texto)
df = df.drop(columns=cols_to_remove)
print(f"Removidas {len(cols_to_remove)} colunas originais (mantendo colunas de texto)")

# 6 - Tratamento de variáveis categóricas
print("\nTratando variáveis categóricas...")

# 6.1 - Label Encoding para variáveis ordinais
ordinal_cols = [
    '¿Cuál es tu edad?',  # Faixas etárias ordenadas
    '¿Hace quánto tiempo me conoces?',  # Tempo ordenado
    '¿Cuál es tu disponibilidad de tiempo para estudiar inglés?',  # Disponibilidade ordenada
    '¿Cuál es tu sueldo anual? (en dólares)',  # Faixas de salário ordenadas
    '¿Cuánto te gustaría ganar al año?',  # Faixas de salário desejado ordenadas
    '¿Crees que aprender inglés te acercaría más al salario que mencionaste anteriormente?',  # Não/Talvez/Sim
    '¿Crees que aprender inglés puede ayudarte en el trabajo o en tu vida diaria?',  # Não/Talvez/Sim
]

# 6.1.1 - Mapas específicos para garantir a ordenação correta
age_map = {
    '18 años a 24 años': 1,
    '25 años a 34 años': 2,
    '35 años a 44 años': 3,
    '45 años a 54 años': 4,
    'Mas de 54': 5,
    'desconhecido': 0
}

time_map = {
    'Te acabo de conocer a través d...': 0,
    'Te sigo desde hace 1 mes': 1,
    'Te sigo desde hace 3 meses': 2,
    'Te sigo desde hace más de 5 me...': 3,
    'Te sigo desde hace 1 año': 4,
    'Te sigo hace más de 1 año': 5,
    'desconhecido': -1
}

availability_map = {
    'Menos de 1 hora al día': 0,
    '1 hora al día': 1,
    '2 horas al día': 2,
    '3 horas al día': 3,
    'Más de 3 horas al día': 4,
    'desconhecido': -1
}

salary_map = {
    'Menos de US$3000': 1,
    'US$3000 a US$5000': 2,
    'US$5000 o más': 3,
    'US$10000 o más': 4,
    'US$20000 o más': 5,
    'desconhecido': 0
}

desired_salary_map = {
    'Al menos US$ 3000 por año': 1,
    'Más de US$5000 por año': 2,
    'Más de US$10000 por año': 3,
    'Más de US$20000 por año': 4,
    'desconhecido': 0
}

belief_map = {
    'Creo que no...': 0,
    'Tal vez': 1,
    '¡Sí, sin duda!': 2,
    '¡Si por su puesto!': 2,  # Variação ortográfica do sim
    'desconhecido': -1
}

# 6.1.2 - Aplicar mapas específicos para variáveis com ordem conhecida
if '¿Cuál es tu edad?' in df.columns:
    df['age_encoded'] = df['¿Cuál es tu edad?'].map(age_map)
    print("mapping done")

if '¿Hace quánto tiempo me conoces?' in df.columns:
    df['time_known_encoded'] = df['¿Hace quánto tiempo me conoces?'].map(time_map)
    print("mapping done")
if '¿Cuál es tu disponibilidad de tiempo para estudiar inglés?' in df.columns:
    df['availability_encoded'] = df['¿Cuál es tu disponibilidad de tiempo para estudiar inglés?'].map(availability_map)
    print("mapping done")
if '¿Cuál es tu sueldo anual? (en dólares)' in df.columns:
    df['current_salary_encoded'] = df['¿Cuál es tu sueldo anual? (en dólares)'].map(salary_map)
    print("mapping done")
if '¿Cuánto te gustaría ganar al año?' in df.columns:
    df['desired_salary_encoded'] = df['¿Cuánto te gustaría ganar al año?'].map(desired_salary_map)
    print("mapping done")
# Ambas colunas de crença
for col in ['¿Crees que aprender inglés te acercaría más al salario que mencionaste anteriormente?', 
            '¿Crees que aprender inglés puede ayudarte en el trabajo o en tu vida diaria?']:
    if col in df.columns:
        new_col = 'belief_salary_encoded' if 'salario' in col else 'belief_work_encoded'
        df[new_col] = df[col].map(belief_map)
        print("mapping done")

# 6.2 - Agrupar categorias raras em variáveis de alta cardinalidade
nominal_high_cardinality = ['¿Cual es tu país?', '¿Cuál es tu profesión?']

for col in nominal_high_cardinality:
    if col in df.columns:
        # Agrupar categorias raras com base em contagem absoluta
        value_counts = df[col].value_counts()
        threshold = 10  # categorias com menos de 10 ocorrências
        rare_categories = [cat for cat, count in value_counts.items() if count < threshold]
        
        # Criar variável agrupada
        grouped_col = col + '_grouped'
        df[grouped_col] = df[col].apply(lambda x: 'Rare' if x in rare_categories else x)
        print("grouping done")
        
        # Aplicar Label Encoding na variável agrupada
        le = LabelEncoder()
        encoded_col = 'country_encoded' if 'país' in col else 'profession_encoded'
        df[encoded_col] = le.fit_transform(df[grouped_col])
        print("Label encoding done")

# 6.3 - Binary encoding para variáveis binárias
if '¿Cuál es tu género?' in df.columns:
    gender_map = {'Mujer': 1, 'Hombre': 0, 'desconhecido': -1}
    df['gender_encoded'] = df['¿Cuál es tu género?'].map(gender_map)
    print("binary done")

# 6.4 - Tratamento de UTMs
# Identificar colunas UTM
utm_cols = [col for col in df.columns if 'UTM_' in col or 'utm_' in col]

for col in utm_cols:
    # Verificar cardinalidade
    cardinality = df[col].nunique()
    
    if cardinality <= 10:  # Baixa cardinalidade
        # Label Encoding convertendo todos os valores para string
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[col].fillna('unknown').astype(str))
    else:  # Alta cardinalidade
        # Vamos apenas agrupar valores raros
        value_counts = df[col].value_counts()
        threshold = 10
        rare_values = [val for val, count in value_counts.items() if count < threshold]
        df[f'{col}_grouped'] = df[col].apply(lambda x: 'Rare' if x in rare_values else x)
        
        # Aplicar label encoding no agrupamento
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[f'{col}_grouped'].fillna('unknown').astype(str))
print("UTMs done")

# 6.5 - GCLID como indicador binário
if 'GCLID' in df.columns:
    df['has_gclid'] = df['GCLID'].notna().astype(int)

# 8 - Salvar dataset com features engineered (não textuais)
print("\nSalvando dataset com features categóricas e numéricas tratadas...")
df.to_csv("datasets/02_2_data_with_feature_treatment_num_and_cat.csv", index=False)
print(f"Dataset salvo com {df.shape[1]} colunas em 'datasets/02_2_data_with_feature_treatment_num_and_cat.csv'")

# 9 - Documentar conjunto final de features
print("\nDocumentando conjunto final de features:")
feature_types = {
    'Categóricas Codificadas': [col for col in df.columns if '_encoded' in col],
    'Agrupadas': [col for col in df.columns if '_grouped' in col],
    'Features Temporais': ['hour', 'day_of_week', 'month', 'year', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'period_of_day'],
    'Features Binárias': ['has_instagram', 'valid_phone', 'has_gclid'],
    'Outras': [col for col in df.columns if not any(col in group for group in [
        [c for c in df.columns if '_encoded' in c],
        [c for c in df.columns if '_grouped' in c],
        ['hour', 'day_of_week', 'month', 'year', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos', 'period_of_day'],
        ['has_instagram', 'valid_phone', 'has_gclid'],
        ['target']
    ])]
}

for feature_type, cols in feature_types.items():
    # Filtrar apenas colunas que existem no dataframe
    cols = [col for col in cols if col in df.columns]
    if cols:
        print(f"\n{feature_type} ({len(cols)}):")
        for col in cols[:10]:  # Limitar a 10 por categoria para concisão
            print(f"  - {col}")
        if len(cols) > 10:
            print(f"  - ... e mais {len(cols) - 10} features")

print("\nFeature engineering (sem NLP) concluído com sucesso!")

Carregando dataset pré-processado...
Dataset carregado: 106613 linhas, 34 colunas

Removendo colunas irrelevantes (exceto texto)...
Criando features derivadas (não textuais)...
Removidas 3 colunas originais (mantendo colunas de texto)

Tratando variáveis categóricas...
mapping done
mapping done
mapping done
mapping done
mapping done
mapping done
mapping done
grouping done
Label encoding done
grouping done
Label encoding done
binary done
UTMs done

Salvando dataset com features categóricas e numéricas tratadas...
Dataset salvo com 80 colunas em '02_2_data_with_feature_treatment_num_and_cat.csv'

Documentando conjunto final de features:

Categóricas Codificadas (21):
  - age_encoded
  - time_known_encoded
  - availability_encoded
  - current_salary_encoded
  - desired_salary_encoded
  - belief_salary_encoded
  - belief_work_encoded
  - country_encoded
  - profession_encoded
  - gender_encoded
  - ... e mais 11 features

Agrupadas (10):
  - ¿Cual es tu país?_grouped
  - ¿Cuál es tu profes

# Features treatment (text features)
1. Using NLP and other techniques to treat text features

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
import re
import warnings
warnings.filterwarnings('ignore')

# 1 - Carregar dataset pré-processado
print("Carregando dataset pré-processado...")
df = pd.read_csv("datasets/02_2_data_with_feature_treatment_num_and_cat.csv")
print(f"Dataset carregado: {df.shape[0]} linhas, {df.shape[1]} colunas")

# 2 - Identificar colunas de texto para processamento
text_cols = [
    'Cuando hables inglés con fluidez, ¿qué cambiará en tu vida? ¿Qué oportunidades se abrirán para ti?',
    '¿Qué esperas aprender en la Semana de Cero a Inglés Fluido?',
    'Déjame un mensaje',
    '¿Qué esperas aprender en la Inmersión Desbloquea Tu Inglés En 72 horas?'
]

# Filtrar apenas colunas existentes no dataframe
text_cols = [col for col in text_cols if col in df.columns]
print(f"Colunas de texto identificadas: {len(text_cols)}")

if not text_cols:
    print("Nenhuma coluna textual encontrada no dataset. Verifique os nomes das colunas.")
    exit()

# 3 - Pré-processamento para análise de texto
print("\nRealizando pré-processamento do texto...")

# 3.1 - Função para limpar e normalizar texto
def clean_text(text):
    if not isinstance(text, str) or pd.isna(text):
        return ""
    
    # Converter para minúsculas
    text = text.lower()
    # Remover URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    # Remover emails
    text = re.sub(r'\S+@\S+', '', text)
    # Remover pontuação e caracteres especiais, mas manter espaços e letras com acentos
    text = re.sub(r'[^\w\s\á\é\í\ó\ú\ñ\ü]', ' ', text)
    # Remover números
    text = re.sub(r'\d+', '', text)
    # Remover espaços extras
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# 3.2 - Aplicar limpeza de texto
for col in text_cols:
    df[f'{col}_clean'] = df[col].apply(clean_text)

# 4 - Features básicas de texto
print("Extraindo features básicas de texto...")
for col in text_cols:
    # 4.1 - Comprimento do texto
    df[f'{col}_length'] = df[col].str.len()
    
    # 4.2 - Contagem de palavras
    df[f'{col}_word_count'] = df[col].apply(lambda x: len(str(x).split()) if pd.notna(x) else 0)
    
    # 4.3 - Presença de caracteres específicos
    df[f'{col}_has_question'] = df[col].str.contains('\?', regex=True, na=False).astype(int)
    df[f'{col}_has_exclamation'] = df[col].str.contains('!', regex=True, na=False).astype(int)
    
    # 4.4 - Média do tamanho das palavras
    df[f'{col}_avg_word_length'] = df[col].apply(
        lambda x: np.mean([len(w) for w in str(x).split()]) if pd.notna(x) and len(str(x).split()) > 0 else 0
    )

# 5 - Análise de sentimento
print("Realizando análise de sentimento...")
for col in text_cols:
    # Função para análise de sentimento com TextBlob (lidar com exceções)
    def get_sentiment(text):
        try:
            if not isinstance(text, str) or pd.isna(text) or text == '':
                return 0
            return TextBlob(text).sentiment.polarity
        except:
            return 0
    
    # Aplicar análise de sentimento
    df[f'{col}_sentiment'] = df[col].apply(get_sentiment)

# 6.4 - Criar features com base em palavras-chave de motivação
print("\nCriando features baseadas em palavras-chave de motivação...")

# 6.2 - Palavras-chave específicas relacionadas a motivação para aprender inglês
motivation_keywords = {
    'trabajo': 'work', 'empleo': 'work', 'carrera': 'work', 'profesional': 'work', 
    'puesto': 'work', 'laboral': 'work', 'sueldo': 'work', 'trabajar': 'work',
    'viaje': 'travel', 'viajar': 'travel', 'turismo': 'travel', 'países': 'travel', 
    'extranjero': 'travel', 'mundo': 'travel', 'internacional': 'travel',
    'comunicar': 'communication', 'comunicación': 'communication', 'hablar': 'communication', 
    'entender': 'communication', 'expresar': 'communication',
    'estudio': 'education', 'estudiar': 'education', 'universidad': 'education', 
    'curso': 'education', 'aprender': 'education', 'educación': 'education',
    'mejor': 'improvement', 'mejorar': 'improvement', 'crecer': 'improvement', 
    'avanzar': 'improvement', 'progresar': 'improvement', 'desarrollar': 'improvement',
    'oportunidad': 'opportunity', 'futuro': 'opportunity', 'posibilidad': 'opportunity', 
    'chance': 'opportunity', 'opción': 'opportunity'
}

# Agrupar por categoria de motivação
for col in text_cols:
    # Inicializar colunas de categorias de motivação
    for category in set(motivation_keywords.values()):
        df[f'{col}_motiv_{category}'] = 0
    
    # Processar cada linha
    for idx, text in enumerate(df[f'{col}_clean']):
        if not isinstance(text, str) or text == '':
            continue
            
        # Verificar presença de cada palavra-chave
        for keyword, category in motivation_keywords.items():
            if keyword in text:
                df.loc[idx, f'{col}_motiv_{category}'] += 1
    
    # Normalizar por comprimento do texto (para textos não vazios)
    mask = df[f'{col}_word_count'] > 0
    for category in set(motivation_keywords.values()):
        col_name = f'{col}_motiv_{category}'
        df.loc[mask, f'{col_name}_norm'] = df.loc[mask, col_name] / df.loc[mask, f'{col}_word_count']

# REMOVIDO: TF-IDF aplicado ao dataset completo (para evitar vazamento de dados)
# REMOVIDA: Análise de poder discriminativo baseada no target

# 8 - Analisar estatísticas básicas do texto (sem uso de target)
print("\nResultados da análise de texto:")

# 8.1 - Estatísticas de sentimento
for col in text_cols:
    sentiment_col = f'{col}_sentiment'
    if sentiment_col in df.columns:
        sentiment_mean = df[sentiment_col].mean()
        sentiment_std = df[sentiment_col].std()
        print(f"  Sentimento em '{col}': média={sentiment_mean:.4f}, std={sentiment_std:.4f}")

# 8.2 - Categorias de motivação mais frequentes
for col in text_cols:
   motiv_cols = [c for c in df.columns if f'{col}_motiv_' in c and not c.endswith('_norm')]
   if motiv_cols:
       motiv_sums = {c.split('_')[-1]: df[c].sum() for c in motiv_cols}
       sorted_motivs = sorted(motiv_sums.items(), key=lambda x: x[1], reverse=True)
       print(f"\n  Categorias de motivação em '{col}':")
       for category, count in sorted_motivs:
           if count > 0:
               print(f"    - {category}: {count} menções")

# 10 - Salvar dataset com features textuais
print("\nSalvando dataset com features textuais e todas as demais features...")
# Manter todas as colunas, exceto versões "limpas" do texto original
columns_to_drop = [f'{col}_clean' for col in text_cols if f'{col}_clean' in df.columns]
df_final = df.drop(columns=columns_to_drop)

# Salvar dataset final com todas as features
df_final.to_csv("datasets/02_3_data_with_feature_treatment_nlp.csv", index=False)
print(f"Dataset completo salvo ({df_final.shape[1]} colunas) em 'datasets/02_3_data_with_feature_treatment_nlp.csv'")

print("\nProcessamento de features textuais concluído com sucesso!")

Carregando dataset pré-processado...
Dataset carregado: 106613 linhas, 80 colunas
Colunas de texto identificadas: 4

Realizando pré-processamento do texto...
Extraindo features básicas de texto...
Realizando análise de sentimento...

Criando features baseadas em palavras-chave de motivação...

Resultados da análise de texto:
  Sentimento em 'Cuando hables inglés con fluidez, ¿qué cambiará en tu vida? ¿Qué oportunidades se abrirán para ti?': média=0.0017, std=0.0333
  Sentimento em '¿Qué esperas aprender en la Semana de Cero a Inglés Fluido?': média=0.0000, std=0.0437
  Sentimento em 'Déjame un mensaje': média=0.0074, std=0.0734
  Sentimento em '¿Qué esperas aprender en la Inmersión Desbloquea Tu Inglés En 72 horas?': média=0.0003, std=0.0173

  Categorias de motivação em 'Cuando hables inglés con fluidez, ¿qué cambiará en tu vida? ¿Qué oportunidades se abrirán para ti?':
    - work: 65439 menções
    - opportunity: 21541 menções
    - improvement: 21497 menções
    - travel: 18490 menç